## Stacking Ensemble Model

In [1]:
import pandas as pd
import numpy as np

from models.utility import load_input, scale_with_minmax, generate_sequences, get_dates_from_config, handle_outliers, get_params_from_config, convert_seq_to_df, dailydf_to_ts, inverse_transform_minmax, transform_dayofyear
from models.lstm import simpleLSTM
from models.deeplearner import DeepLearner

from solutil import evaluations

from sklearn.linear_model import LinearRegression

### Data Import

In [3]:
df_lstm = pd.read_csv('experiments/stacking_data/lstm_ytest.csv')
df_svr = pd.